In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
class_name = 'chair'

save_res = False
top_out_dir = top_evaluation_dir

boost_samples = 1
special_tag = 'jsd'
# skip = ['train', 'test']
skip = ['train']

In [12]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
evaluator.prepare_gt_grid_variables()

In [9]:
best_by_jsd = {'gmm': 'bneck_128_full_32_gaussians',
               'l_w_gan_large': 1800,
               'r_gan': 1700,
               'l_gan_chamfer': 300,
               'l_gan_emd': 100,
               'l_w_gan_small': 1800
              }

best_by_mmd = {'gmm': 'bneck_128_full_40_gaussians',               
               'l_w_gan_large': 2000,
               'r_gan': 1350,
               'l_gan_chamfer': 300,
               'l_gan_emd': 200,
               'l_w_gan_small': 1700
              }

In [6]:
exp_dict_reduced = stored_synthetic_samples(class_name).copy()

exp_dict_reduced.pop('l_w_gan_large', 'None')
exp_dict_reduced.pop('l_gan_chamfer', 'None')
exp_dict_reduced.pop('l_gan_emd', 'None')
exp_dict_reduced.pop('l_w_gan_small', 'None')
exp_dict_reduced.pop('gmm', 'None')

generators_zoo = exp_dict_reduced.keys()

In [7]:
print generators_zoo

['r_gan']


In [13]:
for best_model, tag in zip([best_by_jsd, best_by_mmd], ['jsd', 'mmd']):
    if tag == 'mmd':
        continue
    print tag    
    for experiment_name in generators_zoo:
        print experiment_name
        top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]
        
        if save_res:
            exp_name = osp.basename(top_sample_dir)
            f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + '_'+ tag + '_' + exp_name + '.txt'), 'w')
        else:            
            f_jsd = sys.stdout

        sample_files = []

        if 'gan' in experiment_name:
            epoch = best_model[experiment_name]
            sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
        elif 'gmm' in experiment_name:
            gaussian_type = best_model[experiment_name]
            sample_files.append(osp.join(top_sample_dir, gaussian_type + '.npz'))
        else:
            assert False
                
        for sample_file in sample_files:
            evaluator.prepare_sample_data(sample_file, boost_samples)
            sample_name = osp.basename(sample_file)[:-len('.npz')]
            print sample_name

        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd, skip=skip)

        f_jsd.flush()
        print
        if save_res:
            f_jsd.close()

jsd
r_gan
epoch_1700
epoch_1700
test 0.171577503229
val 0.17434019356

